In [1]:
import pandas as pd

import sys
sys.path.append('../')
from utils import *

pd.set_option('display.max_columns', None)

In [2]:
WYSCOUT_ID = 5414111
SKILLCORNER_ID = 952209

In [3]:
DATA_PATH = f'../../data/networks/match_{SKILLCORNER_ID}/'
PASSES_DF_PATH = DATA_PATH + 'passes_df.pkl'

# Passes Network

In [4]:
passes_df = pd.read_pickle(PASSES_DF_PATH)

Keep only tracking data of players of the opposing team, with responsibility greater than 0.

In [5]:
passes_df = passes_df[~passes_df['tracking.is_teammate']]
passes_df = passes_df[passes_df['responsibility'] > 0]

In [6]:
passes_df.rename(columns={'matchPeriod': 'match.period'}, inplace=True)

In [7]:
passes_df.sample(5)

,match.period,team.name,player.id.wyscout,player.id.skillcorner,pass.recipient.id.wyscout,pass.recipient.id.skillcorner,location.x,location.y,pass.endLocation.x,pass.endLocation.y,play_direction,dxt,tracking.object_id,tracking.x,tracking.y,tracking.z,tracking.is_self,tracking.is_teammate,responsibility
frame,,,,,,,,,,,,,,,,,,,
14301,1,France,209117,277,352988,11516,10.588235,-10.382222,9.529412,-20.160000,BOTTOM_TO_TOP,-0.000343,10738,12.72,-1.14,0.0,False,False,0.039297
33425,2,Netherland,116,3837,370,10738,9.529412,-2.113846,18.000000,0.704615,BOTTOM_TO_TOP,-0.002146,16320,19.93,3.50,0.0,False,False,0.465204
15913,1,Netherland,491458,24466,134102,7863,4.235294,-17.104444,-16.941176,-28.901053,TOP_TO_BOTTOM,-0.007856,946,-1.99,-10.04,0.0,False,False,0.061091
19877,1,France,352988,11516,344132,11606,14.823529,-7.046154,-5.294118,-29.629474,BOTTOM_TO_TOP,0.005184,3837,5.56,-11.78,0.0,False,False,0.461106
13676,1,Netherland,370,10738,491458,24466,18.000000,-25.258947,18.000000,9.771111,TOP_TO_BOTTOM,0.005359,6028,13.22,10.19,0.0,False,False,0.260084


# Passes Network

In [8]:
column_mask = ['player.id.skillcorner', 'pass.recipient.id.skillcorner', 'location.x', 'location.y' , 'pass.endLocation.x', 'pass.endLocation.y', 'dxt', 'responsibility', 'match.period', 'team.name']
group_column_mask = [col for col in column_mask if col != 'responsibility']
passes_network = passes_df[column_mask].groupby(group_column_mask).agg({'responsibility': 'sum'}).reset_index()[column_mask]

In [9]:
passes_network.sample(5)

,player.id.skillcorner,pass.recipient.id.skillcorner,location.x,location.y,pass.endLocation.x,pass.endLocation.y,dxt,responsibility,match.period,team.name
74,5922,12218,-37.833333,29.629474,-24.352941,23.073684,0.001109,1.243659,2,France
289,10738,24466,26.470588,-0.704615,22.235294,12.215556,-0.006741,0.136212,1,Netherland
576,31930,31930,-11.647059,-28.901053,-13.764706,-25.987368,-0.000309,1.269237,1,Netherland
197,10082,7017,-6.352941,20.160000,-3.176471,10.382222,-0.000640,0.326318,2,Netherland
129,7387,7863,-13.764706,-23.802105,-12.705882,-12.826667,0.000875,2.178083,1,Netherland


# Defender Responsibility Network

In [10]:
tracking_object_id = 11606

player_passes_df = passes_df[passes_df['tracking.object_id'] == tracking_object_id]
defender_responsibility_network = player_passes_df[player_passes_df['responsibility'] > 0]

column_mask = ['player.id.skillcorner', 'pass.recipient.id.skillcorner', 'location.x', 'location.y' , 'pass.endLocation.x', 'pass.endLocation.y', 'dxt', 'responsibility', 'match.period', 'team.name']
defender_responsibility_network = defender_responsibility_network[column_mask].reset_index(drop=True)

In [11]:
defender_responsibility_network.sample(5)

,player.id.skillcorner,pass.recipient.id.skillcorner,location.x,location.y,pass.endLocation.x,pass.endLocation.y,dxt,responsibility,match.period,team.name
17,24466,7387,49.750000,25.987368,8.470588,25.258947,-0.063852,0.626318,1,Netherland
16,3837,7387,-36.000000,12.826667,-40.583333,15.271111,-0.000247,0.571114,1,Netherland
12,3837,7017,-3.176471,20.888421,5.294118,18.326667,0.004011,0.089933,1,Netherland
11,7017,3837,7.411765,20.888421,-3.176471,20.888421,-0.004749,0.022303,1,Netherland
40,24466,10082,-5.294118,23.073684,-21.176471,31.086316,-0.003824,0.065480,1,Netherland


# Aggregate Statistics for each Defender

In [12]:
defender_stats = passes_df.groupby('tracking.object_id').agg(
    average_responsibility=('responsibility', 'mean'),
    average_dxt=('dxt', 'mean'),
    group_size=('responsibility', 'size')
).reset_index()

defender_stats.sample(5)

,tracking.object_id,average_responsibility,average_dxt,group_size
31,33512,0.253309,0.005026,15
26,16320,0.273005,0.000230,62
25,14407,0.296403,-0.003915,33
15,10916,0.369854,-0.000709,66
4,3837,0.365994,-0.004813,42


# Defender Dyads

In [13]:
passes_df_cp = passes_df.reset_index()
pass_filt_df = passes_df_cp.groupby('frame').filter(lambda x: len(x) >= 2)

relevant_cols = ['frame', 'tracking.object_id_x', 'tracking.object_id_y',  'dxt_x', 'responsibility_x', 'responsibility_y']
joint_df = pass_filt_df.merge(pass_filt_df, on=['frame'])[relevant_cols]
joint_df = joint_df[joint_df['tracking.object_id_x'] < joint_df['tracking.object_id_y']]
joint_df['joint_resp'] = joint_df['responsibility_x'] + joint_df['responsibility_y']
joint_df.rename(columns={'dxt_x': 'dxt'}, inplace=True)

In [14]:
columns_joint_group = ['frame', 'tracking.player.id.skillcorner_x', 'tracking.player.id.skillcorner_y', 'joint_resp']

defender_dyads_network = joint_df.groupby(['tracking.object_id_x', 'tracking.object_id_y']).agg(
    joint_responsibility_mean=('joint_resp', 'mean'),
    joint_responsibility_sum=('joint_resp', 'sum'),
    group_size=('joint_resp', 'size')
    ).reset_index()

In [15]:
defender_dyads_network.sample(5)

,tracking.object_id_x,tracking.object_id_y,joint_responsibility_mean,joint_responsibility_sum,group_size
3,277,7604,0.561440,2.245759,4
125,10916,31930,0.696419,12.535543,18
18,946,11606,0.627012,6.897129,11
130,11516,12229,0.784254,3.137017,4
64,6028,11516,0.537367,3.761572,7
